In [110]:
import os
import json
import csv

In [111]:
# load configs
with open(os.path.join(os.path.abspath(''), "Configs", "configs.json"), "r") as json_file:
    configs = json.load(json_file)

In [112]:
# load results
results = []
with open(os.path.join(os.path.abspath(''), "results.csv"), newline='') as csv_file:
    reader = csv.DictReader(csv_file, delimiter=';')
    for row in reader:
        results.append(row)

len(results)

148800

In [113]:
# clear empty feature results
def clear_empty_feature_results(results):
    cleared_results = []
    for result in results:
        if result['lpc_weight'] == '0' and result['lpcc_weight'] == '0' and result['mfcc_weight'] == '0' and result['delta_mfcc_weight'] == '0':
            continue
        cleared_results.append(result)
    return cleared_results

results = clear_empty_feature_results(results)

len(results)

148800

In [114]:
# clear empty configs
def clear_empty_configs(configs):
    cleared_configs = []
    for config in configs:
        if config['lpc_weight'] == 0 and config['lpcc_weight'] == 0 and config['mfcc_weight'] == 0 and config['delta_mfcc_weight'] == 0:
            continue
        cleared_configs.append(config)
    return cleared_configs

configs = clear_empty_configs(configs)

len(configs)

480

In [115]:
# get config for result
def get_config_for_result(result):
    # amount_of_frames;size_of_frame;lpc_weight;lpc_order;mfcc_weight;mfcc_order;lpcc_weight;lpcc_order;delta_mfcc_weight;delta_mfcc_order
    for config in configs:
        if (
            config['amount_of_frames'] == int(result['amount_of_frames']) and
            config['size_of_frame'] == int(result['size_of_frame']) and
            config['lpc_weight'] == int(result['lpc_weight']) and
            config['lpc_order'] == int(result['lpc_order']) and
            config['mfcc_weight'] == int(result['mfcc_weight']) and
            config['mfcc_order'] == int(result['mfcc_order']) and
            config['lpcc_weight'] == int(result['lpcc_weight']) and
            config['lpcc_order'] == int(result['lpcc_order']) and
            config['delta_mfcc_weight'] == int(result['delta_mfcc_weight']) and
            config['delta_mfcc_order'] == int(result['delta_mfcc_order'])
        ):
            return config

In [116]:
# get results for config
def get_results_for_config(config):
    matched_results = []
    for result in results:
        if (
            config['amount_of_frames'] == int(result['amount_of_frames']) and
            config['size_of_frame'] == int(result['size_of_frame']) and
            config['lpc_weight'] == int(result['lpc_weight']) and
            config['lpc_order'] == int(result['lpc_order']) and
            config['mfcc_weight'] == int(result['mfcc_weight']) and
            config['mfcc_order'] == int(result['mfcc_order']) and
            config['lpcc_weight'] == int(result['lpcc_weight']) and
            config['lpcc_order'] == int(result['lpcc_order']) and
            config['delta_mfcc_weight'] == int(result['delta_mfcc_weight']) and
            config['delta_mfcc_order'] == int(result['delta_mfcc_order'])
        ):
            matched_results.append(result)

    return matched_results

In [117]:
for config in configs:
    all_results_for_config = get_results_for_config(config)
    # only the first 300 results are relevant
    config['results'] = all_results_for_config[:300]

In [118]:
# check if there are always 300 results for each config
false_configs = []
for config in configs:
    if len(config['results']) != 300:
        print(f"Config {config['id']} has {len(config['results'])} results")
        false_configs.append(config)

len(false_configs)

0

In [119]:
# new results (with only the first 300 results for each config)
new_results = []
for config in configs:
    for result in config['results']:
        new_results.append(result)

results = new_results
len(results)

144000

In [120]:
# get absolute accuracy of result
def get_absolute_accuracy(result) -> float:

    correct_speaker = result['correct_speaker_id']
    accuracy = result[f'speaker_{correct_speaker}']
    return float(accuracy)

In [121]:
test_accuracy = get_absolute_accuracy(results[0])
print(test_accuracy)

0.0


In [122]:
# check which amount_of_frames is better (10000 or 15000)
results_with_10000_frames = []
results_with_15000_frames = []
for result in results:
    if result['amount_of_frames'] == '10000':
        results_with_10000_frames.append(result)
    elif result['amount_of_frames'] == '15000':
        results_with_15000_frames.append(result)
    else:
        print('Error: amount_of_frames is not 10000 or 15000')

avg_accuracy_10000_frames = 0
avg_accuracy_15000_frames = 0

for result in results_with_10000_frames:
    avg_accuracy_10000_frames += get_absolute_accuracy(result)

for result in results_with_15000_frames:
    avg_accuracy_15000_frames += get_absolute_accuracy(result)

avg_accuracy_10000_frames /= len(results_with_10000_frames)
avg_accuracy_15000_frames /= len(results_with_15000_frames)

avg_accuracy_10000_frames, avg_accuracy_15000_frames


(0.41184821371728525, 0.4234793638917221)

In [123]:
results = results_with_15000_frames

In [124]:
# check which size_of_frame is better (400 or 600)
results_with_400_frames = []
results_with_600_frames = []
for result in results:
    if result['size_of_frame'] == '400':
        results_with_400_frames.append(result)
    elif result['size_of_frame'] == '600':
        results_with_600_frames.append(result)
    else:
        print('Error: size_of_frame is not 400 or 600')

avg_accuracy_400_frames = 0
avg_accuracy_600_frames = 0

for result in results_with_400_frames:
    avg_accuracy_400_frames += get_absolute_accuracy(result)

for result in results_with_600_frames:
    avg_accuracy_600_frames += get_absolute_accuracy(result)

avg_accuracy_400_frames /= len(results_with_400_frames)
avg_accuracy_600_frames /= len(results_with_600_frames)

avg_accuracy_400_frames, avg_accuracy_600_frames

(0.4135228420659765, 0.4334358857174818)

In [125]:
results = results_with_600_frames

In [128]:
# get best config (avg accuracy)
best_config = None
best_accuracy = 0
for config in configs:

    avg_accuracy = 0
    for result in config['results']:
        avg_accuracy += get_absolute_accuracy(result)
    avg_accuracy /= len(config['results'])
    if avg_accuracy > best_accuracy:
        best_accuracy = avg_accuracy
        best_config = config

best_config["id"], best_accuracy

(472, 0.8032763993835045)